In [ ]:
import numpy as np
from importlib import reload
import pandas as pd
import fdc
import tqdm
import matplotlib.pyplot as plt
import sleep_stage_function as ssf
fdc=reload(fdc)
ssf=reload(ssf)




# REAL DATA

In [ ]:
# load the data
sleep_tag = pd.read_csv('../Data_for_analysis/EPCTL06.csv')



In [ ]:
sleep_data = np.load('../Data_for_analysis/EPCTL06-prep-001.npy')

In [ ]:
sleep_tag['L'].unique()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Parametri
fs = 128
samples_per_epoch = fs * 30  # 3840

# Espandi labels
stages = sleep_tag['L'].values
expanded_labels = np.repeat(stages, samples_per_epoch)
n_samples = min(len(expanded_labels), sleep_data.shape[1])
expanded_labels = expanded_labels[:n_samples]

# TROVA DOVE FINISCONO LE 'L' INIZIALI
first_non_L = np.where(expanded_labels != 'L')[0]
if len(first_non_L) > 0:
    start_idx = first_non_L[0]
    print(f"Primo sample non-'L' è al sample {start_idx}")
    print(f"Questo corrisponde a {start_idx/fs:.1f} secondi = {start_idx/fs/60:.1f} minuti")
else:
    start_idx = 0

# PLOT A PARTIRE DAL PRIMO SAMPLE VERO
n_plot = min(500000, n_samples - start_idx)
plot_start = start_idx
plot_end = start_idx + n_plot

time_seconds = np.arange(n_plot) / fs

fig, ax = plt.subplots(figsize=(16, 5))

# Plot serie temporale
ax.plot(time_seconds, sleep_data[0, plot_start:plot_end], 
        'k-', linewidth=0.3, alpha=0.7)

# Colori per sleep stages
stage_colors = {
    'W': '#FFD700',    # Wake - giallo
    'N1': '#87CEEB',   # N1 - azzurro
    'N2': '#4682B4',   # N2 - blu
    'N3': '#191970',   # N3 - blu scuro
    'R': '#FF6347',    # REM - rosso
    'L': '#D3D3D3',    # Latency - grigio
}

# Background colorato
current_stage = expanded_labels[plot_start]
segment_start = 0

for i in range(1, n_plot + 1):
    idx = plot_start + i
    if idx >= plot_end or expanded_labels[idx] != current_stage:
        color = stage_colors.get(current_stage, '#CCCCCC')
        ax.axvspan(segment_start, (i-1)/fs, alpha=0.35, color=color)
        
        if idx < plot_end:
            current_stage = expanded_labels[idx]
            segment_start = i/fs

# Legenda
stage_order = ['W', 'N1', 'N2', 'N3', 'R', 'L']
stage_names = {
    'W': 'Wake',
    'N1': 'NREM Stage 1',
    'N2': 'NREM Stage 2', 
    'N3': 'NREM Stage 3',
    'R': 'REM',
    'L': 'Latency/Unknown'
}

unique_stages = [s for s in stage_order 
                 if s in np.unique(expanded_labels[plot_start:plot_end])]
legend_elements = [
    Patch(facecolor=stage_colors[stage], 
          label=stage_names[stage], 
          alpha=0.35)
    for stage in unique_stages
]
ax.legend(handles=legend_elements, title='Sleep Stage', 
         loc='upper right', framealpha=0.95, fontsize=10)

ax.set_xlabel('Time (seconds)', fontsize=11)
ax.set_ylabel('EEG amplitude (μV)', fontsize=11)
ax.set_title(f'Sleep EEG (starting from sample {start_idx}, t={start_idx/fs/60:.1f} min)', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

# Statistiche complete
print(f"\n=== STATISTICHE COMPLETE ===")
print(f"Total recording: {n_samples/fs/60:.1f} minutes")
print(f"\nStage distribution (tutta la registrazione):")
for stage in stage_order:
    count = np.sum(expanded_labels == stage)
    percentage = count / len(expanded_labels) * 100
    minutes = count / fs / 60
    if count > 0:
        print(f"  {stage_names[stage]:20s}: {minutes:6.1f} min ({percentage:5.1f}%)")

In [ ]:
# Extract first N1 segment (complet
n1_timeseries, start, end, info = ssf.extract_sleep_stage_segment(sleep_data, 
                                                              sleep_tag, 
                                                              'N1', 
                                                              segment_number=2, 
                                                              segment_length=2,
                                                              skip_first_epoch=True)
    

In [ ]:
#from sleep_stage_function import *

# Esempio 2: Estrai il secondo segmento N3
n3_timeseries, start, end, info = ssf.extract_sleep_stage_segment(
    sleep_data, sleep_tag, 'N3', segment_number=1, fs=128
)


In [ ]:
# Frequenze da analizzare 
freqs = np.linspace(0.5, 45, 500)  # Frequencies from 0.5 to 30 Hz
#freqs = np.linspace(2, 15, 100)  # Da delta a beta basso
omegas = 2 * np.pi * freqs

timeseries = n3_timeseries

# Calcola FDC matrix per il segmento n1
import tqdm

C_freq = np.zeros((timeseries.shape[0], timeseries.shape[0], len(freqs)), dtype=complex)
S_ii = []

for i, f in enumerate(tqdm.tqdm(freqs, desc="Computing FDC for n3")):
    cif = fdc.correlation_freq(
        timeseries,
        time_step=1/fs,
        frequency=f, # in hertz
        n_chunks=50,
        corr_type="covariance"
    )
    C_freq[:, :, i] = cif
    S_ii.append(np.mean(np.diag(cif)))


In [ ]:
reshuffled_ts = fdc.reshuffling(timeseries)
C_freq_shuff = np.zeros((reshuffled_ts.shape[0], reshuffled_ts.shape[0], len(freqs)), dtype=complex)
S_ii_shuff = []

for i, f in enumerate(tqdm.tqdm(freqs, desc=f"Computing FDC")):
    cif = fdc.correlation_freq(
        reshuffled_ts,
        time_step=1/fs,
        frequency=f, # in hertz
        n_chunks=50,
        corr_type="covariance"
    )
    C_freq_shuff[:, :, i] = cif
    S_ii_shuff.append(np.mean(np.diag(cif)))


In [ ]:
phaserand_ts = fdc.phase_randomization(timeseries)
C_freq_phaserand = np.zeros((phaserand_ts.shape[0], phaserand_ts.shape[0], len(freqs)), dtype=complex)
S_ii_phaserand = []

for i, f in enumerate(tqdm.tqdm(freqs, desc=f"Computing FDC")):
    cif = fdc.correlation_freq(
        phaserand_ts,
        time_step=1/fs,
        frequency=f, # in hertz
        n_chunks=50,
        corr_type="covariance"
    )
    C_freq_phaserand[:, :, i] = cif
    S_ii_phaserand.append(np.mean(np.diag(cif)))

In [ ]:
blockperm_ts = fdc.block_permutation(timeseries)
C_freq_blockperm = np.zeros((blockperm_ts.shape[0], blockperm_ts.shape[0], len(freqs)), dtype=complex)
S_ii_blockperm = []

for i, f in enumerate(tqdm.tqdm(freqs, desc=f"Computing FDC")):
    cif = fdc.correlation_freq(
        blockperm_ts,
        time_step=1/fs,
        frequency=f, # in hertz
        n_chunks=50,
        corr_type="covariance"
    )
    C_freq_blockperm[:, :, i] = cif
    S_ii_blockperm.append(np.mean(np.diag(cif)))

In [ ]:
plt.figure(figsize=(12, 6))
plt.loglog(freqs, S_ii, linewidth=2, label='Power spectrum')
plt.loglog(freqs, S_ii_shuff, linewidth=1.5, linestyle='--', label='Power spectrum (reshuffled)')
plt.loglog(freqs, S_ii_phaserand, linewidth=1.5, linestyle=':', label='Power spectrum (phase randomized)')
plt.loglog(freqs, S_ii_blockperm, linewidth=1.5, linestyle='-.', label='Power spectrum (block permutation)')

# Bands
plt.axvspan(0.5, 4, alpha=0.2, color='blue', label='Delta (0.5-4 Hz)')
plt.axvspan(4, 8, alpha=0.2, color='orange', label='Theta (4-8 Hz)')
plt.axvspan(8, 13, alpha=0.3, color='green', label='Alpha (8-13 Hz)')
plt.axvspan(13, 30, alpha=0.2, color='red', label='Beta (13-30 Hz)')
plt.axvspan(30, 100, alpha=0.2, color='purple', label='Gamma (30-100 Hz)')

plt.xlabel('Frequency (Hz)', fontsize=12)
plt.ylabel('Power', fontsize=12)
plt.title('Power Spectrum N3', fontsize=13, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.savefig('power_spectrum_with_bands.png')
plt.show()

## Select a chunk of each of the 4 sleep phases and compare the 4 power spectrum

In [ ]:
selected_timeseries = {}
freqs = np.linspace(0.3, 45, 1000)  # Frequencies from 0.3 to 45 Hz

C_freqs_dict = {}
S_ii_dict = {}

for stage in ['W', 'N1', 'N2', 'N3', 'R']:

    timeseries, start, end, info = ssf.extract_sleep_stage_segment(
        sleep_data, sleep_tag, stage, segment_number=0, segment_length=-1, skip_first_epoch=True, fs=fs
        )
    selected_timeseries[stage] = timeseries

    C_freq = np.zeros((timeseries.shape[0], timeseries.shape[0], len(freqs)), dtype=complex)
    S_ii = []
    
    for i, f in enumerate(tqdm.tqdm(freqs, desc=f"Computing FDC for {stage}")):
        cif = fdc.correlation_freq(
            timeseries,
            time_step=1/fs,
            frequency=f, # in hertz
            n_chunks=50,
            corr_type="covariance"
        )
        C_freq[:, :, i] = cif
        S_ii.append(np.mean(np.diag(cif)))

    C_freqs_dict[stage] = C_freq
    S_ii_dict[stage] = S_ii

    

### baseline 

In [ ]:

C_freqs_shuff = {}
S_ii_shuff = {}

for stage in ['W', 'N1', 'N2', 'N3', 'R']:
    reshuffled_ts = fdc.reshuffling(selected_timeseries[stage])
    C_freq = np.zeros((reshuffled_ts.shape[0], reshuffled_ts.shape[0], len(freqs)), dtype=complex)
    S_ii = []
    
    for i, f in enumerate(tqdm.tqdm(freqs, desc=f"Computing FDC for {stage}")):
        cif = fdc.correlation_freq(
            reshuffled_ts,
            time_step=1/fs,
            frequency=f, # in hertz
            n_chunks=50,
            corr_type="covariance"
        )
        C_freq[:, :, i] = cif
        S_ii.append(np.mean(np.diag(cif)))

    C_freqs_shuff[stage] = C_freq
    S_ii_shuff[stage] = S_ii

In [ ]:
freqs = np.linspace(0.3, 45, 1000)  # Frequencies from 0.3 to 45 Hz

In [ ]:
for stage in S_ii_dict.keys():
    plt.figure(figsize=(12, 6))
    plt.loglog(freqs, S_ii_dict[stage], linewidth=2, label='Power spectrum')
    plt.loglog(freqs, S_ii_shuff[stage], linewidth=1, linestyle='--', label='Baseline (reshuffled)')

    # Barre verticali per alpha band (8-13 Hz)
   

    # Opzionale: aggiungi anche altre bande
    plt.axvspan(0.3, 5, alpha=0.2, color='blue', label='Delta (0.5-5 Hz)')
    plt.axvspan(5, 8, alpha=0.2, color='orange', label='Theta (5-8 Hz)')
    plt.axvspan(8, 13, alpha=0.2, color='green', label='Alpha (8-13 Hz)')
    plt.axvspan(12, 15, alpha=0.2, color='red', label='Sigma (12-15 Hz)')
    plt.axvspan(13, 30, alpha=0.2, color='purple', label='Beta (13-30 Hz)')
    plt.axvspan(30, 45, alpha=0.2, color='magenta', label='Gamma (30-100 Hz)')

    plt.xlabel('Frequency (Hz)', fontsize=12)
    plt.ylabel('Power', fontsize=12)
    plt.title(f'Power Spectrum for {stage}', fontsize=13, fontweight='bold')
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3, which='both')
    plt.tight_layout()
    plt.savefig(f'power_spectrum_with_bands_{stage}.png')
    plt.show()